In [1]:
import numpy
import tensorflow

import matplotlib.pyplot as plt
from db.driver import Driver

%matplotlib inline 

In [2]:
driver = Driver()

In [3]:
driver.catagory_list()

[u'diamond', u'cat', u'airplane', u'flower', u'basketball', u'envelope']

In [4]:
driver.total_size()

819886

In [5]:
{ catagory: driver.size_of_catagory(catagory) for catagory in driver.catagory_list()}

{u'airplane': 151623,
 u'basketball': 133793,
 u'cat': 123202,
 u'diamond': 131587,
 u'envelope': 134863,
 u'flower': 144818}